In [80]:
import pandas as pd
from utils import *
import re
regexp = re.compile(r'фоп|фізична особа підприємець')

In [57]:
test_df = pd.read_excel('Data/Auxillary/Список закладів.xlsx', skiprows=1)
test_df['legal_entity_edrpou'] = test_df['legal_entity_edrpou'].apply(lambda x: x if (x is pd.NA or x == 'ФОП') else str(x).rjust(8, '0'))
test_df = test_df[test_df['care_type']=='Первинна']
test_df

,legal_entity_id,legal_entity_name,legal_entity_edrpou,care_type,property_type,legal_entity_email,legal_entity_website,legal_entity_phone,legal_entity_owner_name,registration_area,registration_settlement,registration_address,lat,lng
2,6e6b9d4e-231d-4b6a-a8d7-081f055fe8d4,КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО УСАТІВСЬК...,38617509,Первинна,Комунальна,linarudoman@gmail.com,NaN,+380485225112,Рудоманенко Олена Вікторівна,ОДЕСЬКА,УСАТОВЕ,"Україна, 67663, Одеська обл., Одеський р-н, се...",46.532867,30.657932
3,c2b00dbc-bbb9-4b8c-a4df-7a409e5c575f,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ...",37934309,Первинна,Комунальна,ugledariao@gmail.com,NaN,+380627364093,Сосюра Наталія Миколаївна,ДОНЕЦЬКА,ВУГЛЕДАР,"Україна, 85670, Донецька обл., місто Вугледар,...",47.775308,37.239382
4,a2e144ce-0307-4c6e-bdb9-6186d519a531,МІСЬКО ВІТА ВАСИЛІВНА,ФОП,Первинна,ФОП,vt.klinik@gmail.com,NaN,+380506906522,Місько Віта Василівна,ЧЕРНІВЕЦЬКА,ЗАСТАВНА,"Україна, 59400, Чернівецька обл., Заставнівськ...",48.531439,25.833238
5,ba94b288-1b59-4f58-9f0c-117b3f8ac7f9,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""МЕДИЧ...",37691162,Первинна,Приватна (без ФОП),moe_zdorovie@ukr.net,NaN,+380974631203,Хмільовий Дмитро Андрійович,ВІННИЦЬКА,ВІННИЦЯ(З),"Україна, 21000, Вінницька обл., місто Вінниця(...",49.236732,28.412584
6,eb1b47d6-d116-446f-bfd4-d000884574ca,ЯКИМЧУК НАТАЛІЯ ВАЛЕРІЇВНА,ФОП,Первинна,ФОП,n.yakimchuk@homedoctor.in.ua,homedoctor.in.ua,+380637000640,Якимчук Наталія Валеріївна,ОДЕСЬКА,ОДЕСА,"Україна, 65000, Одеська обл., місто Одеса, ВУЛ...",46.578154,30.799197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,8fdb1613-3c19-491c-bdbb-7a124cb5f8c7,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ДОБРОПІЛ...",37755220,Первинна,Комунальна,cpmsd.dobr@ukr.net,https://dcpmsd.dn.ua,+380627725303,Козенятко Ірина Іванівна,ДОНЕЦЬКА,ДОБРОПІЛЛЯ,"Україна, 85000, Донецька обл., місто Добропілл...",48.462386,37.079921
4115,a065e232-e36f-408f-a9e0-60243a584c00,КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО МИКОЛАЇВ...,38458175,Первинна,Комунальна,cpmsd7_office@ukr.net,https://cpmsd7.lic.org.ua,"+380678073761, +380512640816",Власова Олена Миколаївна,МИКОЛАЇВСЬКА,МИКОЛАЇВ,"Україна, 54051, Миколаївська обл., місто Микол...",46.856015,32.017890
4116,5c6d9036-7a55-40de-aa86-b5c355303bbb,ПАВЛЕНКО НАТАЛІЯ ОЛЕКСІЇВНА,ФОП,Первинна,ФОП,nata.pa1976@gmail.com,NaN,+380956281094,Павленко Наталія Олексіївна,ХАРКІВСЬКА,БОРОВА,"Україна, 63801, Харківська обл., Ізюмський р-н...",49.374251,37.623014
4119,cb19377d-8c28-4f75-b1b4-d6cf243c0622,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""УКРАЇ...",31066166,Первинна,Приватна (без ФОП),admin@unicam.com.ua,unicam.com.ua,+380500303034,Білецький Павло Степанович,СУМСЬКА,СУМИ,"Україна, 40034, Сумська обл., місто Суми, ПРОС...",50.914314,34.833996


In [58]:
test_df['legal_entity_name'].nunique()

2479

In [59]:
test_edrdou_set = set(test_df['legal_entity_edrpou'].unique())

test_df['legal_entity_edrpou'].str.len().value_counts()

legal_entity_edrpou
8    1660
3     819
Name: count, dtype: int64

In [123]:
df = pd.read_excel('Data/ВСЬОГО – 2024-02-06.xlsx', sheet_name='Залишки поточні', dtype={' код ЄДРПОУ': str}, engine='openpyxl')
df.columns = df.columns.str.strip()
df.rename(columns={'Серія препарту': 'Серія препарату'}, inplace=True)
# ———————————————————————————— Визначити назву вакцини за серією препарату і переписати назви у вхідних даних ————————————————————————————
series_df = df.groupby(['Серія препарату'])['Міжнародна непатентована назва'].apply(lambda vaccines: [[vaccine, len(vaccines[vaccines==vaccine])] for vaccine in vaccines.unique() if not pd.isna(vaccine)]).reset_index()

# Залишити тільки серії, де є хоча б одна назва препарату
series_df = series_df[~(series_df['Міжнародна непатентована назва'].apply(lambda vaccines: len(vaccines))==0)]

# Обрати назву препарату, яка найчастіше зустрічається в серії
overwritings = series_df.apply(lambda row: max(row['Міжнародна непатентована назва'], key=lambda pair: pair[1])[0], axis=1)
series_df['Назва препарату'] = overwritings

series_df = series_df[['Серія препарату', 'Назва препарату']].set_index('Серія препарату')

df['Міжнародна непатентована назва'] = df[['Серія препарату', 'Міжнародна непатентована назва']].apply(lambda row: series_df.to_dict()['Назва препарату'].get(row['Серія препарату'], row['Міжнародна непатентована назва']), axis=1)
# ————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
df['Міжнародна непатентована назва'] = df['Міжнародна непатентована назва'].replace(vaccine_shorts)

# Remove vaccines with no Facility or EDRPOU
facilitiles_with_no_edrpou_set = set(df[(df['код ЄДРПОУ']=='') | (df['код ЄДРПОУ'].isna())]['Заклад'].dropna().unique()) - set(df[~((df['код ЄДРПОУ']=='') | (df['код ЄДРПОУ'].isna()))]['Заклад'].dropna().unique())
facilitiles_with_no_edrpou_df = df[df['Заклад'].isin(facilitiles_with_no_edrpou_set)]
df = df[ ~((df['Заклад'].isna() | (df['Заклад'].str.len() == 0)) & (df['код ЄДРПОУ'].isna() | (df['код ЄДРПОУ'].str.len() == 0))) ]

df['код ЄДРПОУ'] = df['код ЄДРПОУ'].str.rjust(8, '0')
df['код ЄДРПОУ'].fillna('', inplace=True)
edrpou_series = df['код ЄДРПОУ']
name_series = df['Заклад']

df

,код ЄДРПОУ,Заклад,Серія препарату,Міжнародна непатентована назва,Торгівельна назва,Виробник,Термін придатності,Джерело фінансування,Кількість доз,дата оновлення,Регіон
0,35527334,"КНП ""ЦПМСД №1"" м. Вінниця",1141M019A,ХІБ,ВАКЦИНА КОН`ЮГОВАНА ДЛЯ ПРОФІЛАКТИКИ ЗАХВОРЮВА...,"СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2024-05-21,Державний бюджет 2021,54,2024-02-02,Вінницька область
1,35527334,"КНП ""ЦПМСД №1"" м. Вінниця",X007788,КПК,"MMRVAXPRO II 0,5ML 10X1DOSE VIAL L25","Merck & Co., Inc.",2024-05-26,Гуманітарна допомога,15,2024-02-02,Вінницька область
2,35527334,"КНП ""ЦПМСД №1"" м. Вінниця",D2708,АДП-М,"ТЕТАДІФ суспензія для ін'єкцій 0,5 мл (1 доза)...",ББ-НЦІПХЛтд. Болгарія,2025-11-30,Державний бюджет 2021,142,2024-02-02,Вінницька область
3,35527334,"КНП ""ЦПМСД №1"" м. Вінниця",2249001B,ІПВ,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V.,2025-12-08,Гуманітарна допомога,30,2024-02-02,Вінницька область
4,35527334,"КНП ""ЦПМСД №1"" м. Вінниця",62С22009А,Сказ,"ІНДІРАБ ВАКЦИНА АНТИРАБІЧНА ОЧИЩЕНА, ІНАКТИВОВ...","ТОВ ""Фарма Лайф"", Україна",2025-10-31,Державний бюджет 2021,7,2024-02-02,Вінницька область
...,...,...,...,...,...,...,...,...,...,...,...
17345,3016503465,ФОП Мирошниченко,221100123A,АКДП,"DTP VACCINE (2023) Adsorbed Diphtheria, Tetanu...","СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2025-01-31,Гуманітарна допомога,10,2024-02-02,Запорізька область
17346,3016503465,ФОП Мирошниченко,X007040,КПК,"MMRVAXPRO II 0,5ML 10X1DOSE VIAL L25","Merck & Co., Inc.",2024-05-26,Гуманітарна допомога,15,2024-02-02,Запорізька область
17347,3016503465,ФОП Мирошниченко,0401622,АДП,"АДСОРБОВАНА ВАКЦИНА ДТ / ADSORBED DT VACCINE, ...","ПТ БІО ФАРМА (Персеро), Індонезія",2025-12-21,Державний бюджет 2021,10,2024-02-02,Запорізька область
17348,3016503465,ФОП Мирошниченко,2310002A,ІПВ,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V. Netherlands,2026-02-14,Гуманітарна допомога,5,2024-02-02,Запорізька область


##### Знайти заклади, які мають багато назв на один ЄДРПОУ

In [117]:
multiple_names_edrpou = df.groupby('код ЄДРПОУ')['Заклад'].apply(lambda x: list(set(x)))
multiple_names_edrpou = multiple_names_edrpou[multiple_names_edrpou.apply(lambda x: len(x)>1)]
multiple_names_edrpou = multiple_names_edrpou.loc[multiple_names_edrpou.index.str.len() > 0].index

multiple_names_df = df[df['код ЄДРПОУ'].isin(multiple_names_edrpou)][['код ЄДРПОУ', 'Регіон', 'Заклад']]
multiple_names_df = multiple_names_df.groupby('код ЄДРПОУ')[['Заклад', 'Регіон']].apply(lambda x: pd.Series(["\n".join(list(x['Заклад'])), x['Регіон'].iloc[0]], index=['Заклад', 'Регіон']))
multiple_names_df

,Заклад,Регіон
код ЄДРПОУ,,
01982583,"КНП ""Липовецька МЛ"" Липовецької МР Віннимцьког...",Вінницька область
01990756,"КНП""БЛІЛ КМР""\nКНП""БЛІЛ КМР""\nКНП""БЛІЛ КМР""\nК...",Донецька область
01993486,"КНП''Тисменицька міська лікарня""Тисменицької м...",Івано-Франківська область
01995290,"КНП ""Петрівська Центральна лікарня"" Петрівсько...",Кіровоградська область
01998360,"КНП ""Березанська ЦРЛ""\nКНП ""Березанская ЦРЛ""\n...",Миколаївська область
...,...,...
43731768,"ТОВ ""КСМ""Біхелсі""""\nТОВ ""КСМ""Біхелсі""""\nТОВ ""К...",Волинська область
43762272,ТОВ МЦ Лікарська династія\nТОВ МЦ Лікарська ди...,Київська область
43778764,"ТОВ ""Медикал плюс""\nТОВ ""Медикал плюс""\nТОВ ""М...",Волинська область


In [128]:
facilitiles_with_no_edrpou_df.groupby("Регіон").apply(lambda x: pd.Series(["\n".join(list(set(x['Заклад'])))], index=['Заклад']))

,Заклад
Регіон,
Івано-Франківська область,"КНП ""ГОРОДЕНКІВСЬКА БАГАТОПРОФІЛЬНА ЛІКАРНЯ ІН..."
Дніпропетровська область,"ФОП Каліта Д.К.\nТОВ ""Перша Приватна Полікліні..."
Закарпатська область,ФОП КРАВЧУК\nКОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМС...
Київська область,"Комунальне некомерційне підприємство ""Березанс..."
Львівська область,КНП «Жовківська лікарня» Жовківської міської р...
Одеська область,ФОП Радайкін Олександр Дмитрович
Полтавська область,Фізична особа-підприємець Ткаченко Ірина Леоні...
Рівненська область,"КНП ""Костопільська багатопрофільна лікарня інт..."
Сумська область,"ФОП Рогаль Л.І.\nКНП""Конотопська ЦРЛ"" КМР\nКНП..."


In [62]:
df[ (df['код ЄДРПОУ'].isin({el for el in edrpou_series.unique() if len(el)!=8})) & ~(df['Заклад'].fillna('').str.lower().str.contains("фоп")) ][['Регіон', 'код ЄДРПОУ', 'Заклад']].drop_duplicates('код ЄДРПОУ').to_excel(
    'Misc/Заклади з неправильними ЄДРПОУ.xlsx', index=False
)

##### Які довжини у кодів ЄДРПОУ у таблиці Стариченка?
> ###### Дозволяє відстежити наявність записів типу "ФОП"

In [63]:
df['код ЄДРПОУ'].drop_duplicates().str.len().value_counts()

код ЄДРПОУ
8     1475
10     332
9        3
0        1
11       1
Name: count, dtype: int64

##### Скільки:
* Унікальних ЄДРПОУ (таблиця Стариченка)
* Унікальних назв (таблиця Стариченка)
* Унікальних ЄДРПОУ (перевірочна таблиця)
* Спільних ЄДРПОУ

In [64]:
len(set(edrpou_series)), len(set(name_series)), len(test_edrdou_set), len(set(edrpou_series) & test_edrdou_set)

(1812, 2183, 1661, 1051)

In [65]:
df['Міжнародна непатентована назва'].value_counts()

Міжнародна непатентована назва
КПК         2397
ГепВ        2130
ІПВ         2099
АДП-М       1834
Пента       1743
АКДП        1538
ХІБ         1373
АДП         1292
БЦЖ          881
ОПВ          856
Сказ         810
ГепВ-Дор     380
Правець        6
Name: count, dtype: int64

In [130]:
# Drop all the duplicate lines with the same EDRPOU (different lines usually represent different responsive persons in the same facility)
facilities_only_test_df = test_df.drop_duplicates('legal_entity_edrpou')
facilities_only_test_df = facilities_only_test_df[facilities_only_test_df['legal_entity_edrpou']!="ФОП"]

result = pd.DataFrame(columns=["Міжнародна непатентована назва", "registration_area", "legal_entity_name", "код ЄДРПОУ", "legal_entity_phone", "legal_entity_email"])
for vaccine, data in df.groupby('Міжнародна непатентована назва'):
    # Pick the data for one specific vaccine and outer join it with the dataset of all the facilities that should have it
    facilities_reporing_vaccine = data.join(facilities_only_test_df.set_index('legal_entity_edrpou'), on='код ЄДРПОУ', how='outer', rsuffix='_test')
    
    # Now, if the vaccine_name field is empty, it means that the facility does not report this vaccine (otherwise it would have been joined by the EDRPOU)
    data_to_add = facilities_reporing_vaccine[facilities_reporing_vaccine['Міжнародна непатентована назва'].isna()][[
        'Міжнародна непатентована назва', "registration_area", "legal_entity_name", "код ЄДРПОУ", "legal_entity_phone", "legal_entity_email"
    ]]
    
    # Fill the vaccine name and add the data to the result
    data_to_add['Міжнародна непатентована назва'] = vaccine
    result = pd.concat([result, data_to_add], ignore_index=True, axis=0)

with pd.ExcelWriter('Misc/Заклади, які не звітують про вакцину.xlsx') as writer:
    for region, data in result.groupby('registration_area'):
        data.to_excel(writer, sheet_name=region, index=False)

    ignorant_facilities = result[~result['Міжнародна непатентована назва'].isin(['ГепВ-Дор', 'Правець', 'ДПКГП'])]

    ignorant_facilities_names = ignorant_facilities.groupby('код ЄДРПОУ')['Міжнародна непатентована назва'].nunique() == 11
    # in pd.Series leave only those index values that are True
    ignorant_facilities_names = ignorant_facilities_names.loc[lambda x: x].index

    ignorant_facilities = ignorant_facilities[ignorant_facilities['код ЄДРПОУ'].isin(ignorant_facilities_names)].drop('Міжнародна непатентована назва', axis=1).drop_duplicates().sort_values('registration_area')
    ignorant_facilities.to_excel(writer, sheet_name="Заклади, які не звітують про жодну вакцину", index=False)
    
    multiple_names_edrpou = df.groupby('код ЄДРПОУ')['Заклад'].apply(lambda x: list(set(x)))
    multiple_names_edrpou = multiple_names_edrpou[multiple_names_edrpou.apply(lambda x: len(x)>1)]
    multiple_names_edrpou = multiple_names_edrpou.loc[multiple_names_edrpou.index.str.len() > 0].index
    
    multiple_names_df = df[df['код ЄДРПОУ'].isin(multiple_names_edrpou)][['код ЄДРПОУ', 'Регіон', 'Заклад']]
    multiple_names_df = multiple_names_df.groupby('код ЄДРПОУ')[['Заклад', 'Регіон']].apply(lambda x: pd.Series([ x['Регіон'].iloc[0], "\n".join(list(set(x['Заклад'])))], index=['Регіон', 'Заклад'])).reset_index()
    
    multiple_names_df.to_excel(writer, sheet_name="Заклади з однаковим ЄДРПОУ|ІПН", index=False)
    
    facilitiles_with_no_edrpou_df.groupby("Регіон").apply(lambda x: pd.Series(["\n".join(list(set(x['Заклад'])))], index=['Заклад'])).reset_index().to_excel(writer, sheet_name="Заклади без ЄДРПОУ", index=False)
    

/Users/andrewlickbrekht/.local/share/virtualenvs/Vaccine_Administration_Analysis-tY9_woho/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


##### Знайти усіх ФОПів, які звітують у таблицю Стариченка і містяться у файлі від ЦГЗ


In [81]:
fops_df = df[df['Заклад'].str.lower().str.contains('фоп')]
fops_df.loc[:, 'Прізвище'] = fops_df['Заклад'].str.lower().str.replace(regexp, '', regex=True).str.strip().str.split(' ').str[0]
fops_df

/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_16443/2165553812.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fops_df.loc[:, 'Прізвище'] = fops_df['Заклад'].str.lower().str.replace(regexp, '', regex=True).str.strip().str.split(' ').str[0]


,код ЄДРПОУ,Заклад,Серія препарату,Міжнародна непатентована назва,Торгівельна назва,Виробник,Термін придатності,Джерело фінансування,Кількість доз,дата оновлення,Регіон,Прізвище
775,3302705015,ФОП Мельник,0321Q018A,ГепВ,"HEPATITIS B VACCENE DNA RECOMBINANT, 1 DOSE VI...","СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2024-11-13,Гуманітарна допомога,2,2024-02-02,Вінницька область,мельник
776,3302705015,ФОП Мельник,1141M019A,ХІБ,ВАКЦИНА КОН`ЮГОВАНА ДЛЯ ПРОФІЛАКТИКИ ЗАХВОРЮВА...,"СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2024-05-21,Державний бюджет 2021,11,2024-02-02,Вінницька область,мельник
777,3302705015,ФОП Мельник,X007788,КПК,"MMRVAXPRO II 0,5ML 10X1DOSE VIAL L25","Merck & Co., Inc.",2024-05-26,Гуманітарна допомога,6,2024-02-02,Вінницька область,мельник
778,3302705015,ФОП Мельник,D2708,АДП-М,"ТЕТАДІФ суспензія для ін'єкцій 0,5 мл (1 доза)...",ББ-НЦІПХЛтд. Болгарія,2025-11-30,Державний бюджет 2021,30,2024-02-02,Вінницька область,мельник
779,3302705015,ФОП Мельник,2249001B,ІПВ,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V.,2025-12-08,Гуманітарна допомога,20,2024-02-02,Вінницька область,мельник
...,...,...,...,...,...,...,...,...,...,...,...,...
17345,3016503465,ФОП Мирошниченко,221100123A,АКДП,"DTP VACCINE (2023) Adsorbed Diphtheria, Tetanu...","СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2025-01-31,Гуманітарна допомога,10,2024-02-02,Запорізька область,мирошниченко
17346,3016503465,ФОП Мирошниченко,X007040,КПК,"MMRVAXPRO II 0,5ML 10X1DOSE VIAL L25","Merck & Co., Inc.",2024-05-26,Гуманітарна допомога,15,2024-02-02,Запорізька область,мирошниченко
17347,3016503465,ФОП Мирошниченко,0401622,АДП,"АДСОРБОВАНА ВАКЦИНА ДТ / ADSORBED DT VACCINE, ...","ПТ БІО ФАРМА (Персеро), Індонезія",2025-12-21,Державний бюджет 2021,10,2024-02-02,Запорізька область,мирошниченко
17348,3016503465,ФОП Мирошниченко,2310002A,ІПВ,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V. Netherlands,2026-02-14,Гуманітарна допомога,5,2024-02-02,Запорізька область,мирошниченко


In [82]:
fops_test_df = test_df[test_df['legal_entity_edrpou'] == 'ФОП']
fops_test_df['Прізвище'] = fops_test_df['legal_entity_name'].str.split(' ').str[0].str.lower()
fops_test_df

/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_16443/3499964528.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fops_test_df['Прізвище'] = fops_test_df['legal_entity_name'].str.split(' ').str[0].str.lower()


,legal_entity_id,legal_entity_name,legal_entity_edrpou,care_type,property_type,legal_entity_email,legal_entity_website,legal_entity_phone,legal_entity_owner_name,registration_area,registration_settlement,registration_address,lat,lng,Прізвище
4,a2e144ce-0307-4c6e-bdb9-6186d519a531,МІСЬКО ВІТА ВАСИЛІВНА,ФОП,Первинна,ФОП,vt.klinik@gmail.com,NaN,+380506906522,Місько Віта Василівна,ЧЕРНІВЕЦЬКА,ЗАСТАВНА,"Україна, 59400, Чернівецька обл., Заставнівськ...",48.531439,25.833238,місько
6,eb1b47d6-d116-446f-bfd4-d000884574ca,ЯКИМЧУК НАТАЛІЯ ВАЛЕРІЇВНА,ФОП,Первинна,ФОП,n.yakimchuk@homedoctor.in.ua,homedoctor.in.ua,+380637000640,Якимчук Наталія Валеріївна,ОДЕСЬКА,ОДЕСА,"Україна, 65000, Одеська обл., місто Одеса, ВУЛ...",46.578154,30.799197,якимчук
8,d0549868-32e3-4c83-88f3-dd938f984b15,СЕЛЕЗНЬОВА СВІТЛАНА ЛЕОНІДІВНА,ФОП,Первинна,ФОП,medkabinet43@gmail.com,NaN,+380980632119,Селезньова Світлана Леонідівна,ОДЕСЬКА,БІЛГОРОД-ДНІСТРОВСЬКИЙ,"Україна, 67700, Одеська обл., місто Білгород-Д...",46.190439,30.342594,селезньова
9,760c6b14-64ea-4886-92d6-7468860e6a63,ДОЛІШНА ОКСАНА ІВАНІВНА,ФОП,Первинна,ФОП,doleshniy.andriy@gmail.com,NaN,+380995113771,Долішна Оксана Іванівна,ІВАНО-ФРАНКІВСЬКА,ІВАНО-ФРАНКІВСЬК(З),"Україна, 76018, Івано-Франківська обл., місто ...",48.919247,24.722991,долішна
10,e61e9f53-3bf8-4b33-bf49-eb37a9c84bec,ЮРЧЕНКО ОКСАНА ВОЛОДИМИРІВНА,ФОП,Первинна,ФОП,dr.oksanayurchenko@gmail.com,NaN,+380992860263,Юрченко Оксана Володимирівна,ПОЛТАВСЬКА,ЛОХВИЦЯ,"Україна, 37200, Полтавська обл., Миргородський...",50.358872,33.259035,юрченко
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4094,303371f3-c58e-4394-bc7d-734970961a61,РАДЬКО ІННА ОЛЕКСАНДРІВНА,ФОП,Первинна,ФОП,innaradko84@gmail.com,NaN,+380991239202,Радько Інна Олександрівна,КІРОВОГРАДСЬКА,КРОПИВНИЦЬКИЙ,"Україна, 25031, Кіровоградська обл., місто Кро...",48.507933,32.262317,радько
4095,6378e55a-4ce1-4ec1-9eb6-0e73b2ecbe56,Філатов Олександр Володимирович,ФОП,Первинна,ФОП,filatovav.home@gmail.com,NaN,+380932463629,Філатов Олександр Володимирович,КИЇВСЬКА,СЛАВУТИЧ(З),"Україна, 07101, Київська обл., місто Славутич(...",51.518644,30.760449,філатов
4111,a9e06381-77e1-4ce3-b66a-c646b1189e8b,ШАРЕНКО-ЛАПТЄВА ЛАРИСА ВОЛОДИМИРІВНА,ФОП,Первинна,ФОП,sharenko.fop@gmail.com,NaN,+380501720659,Шаренко-Лаптєва Лариса Володимирівна,ПОЛТАВСЬКА,ПИРЯТИН,"Україна, 37000, Полтавська обл., Пирятинський ...",50.251287,32.506312,шаренко-лаптєва
4116,5c6d9036-7a55-40de-aa86-b5c355303bbb,ПАВЛЕНКО НАТАЛІЯ ОЛЕКСІЇВНА,ФОП,Первинна,ФОП,nata.pa1976@gmail.com,NaN,+380956281094,Павленко Наталія Олексіївна,ХАРКІВСЬКА,БОРОВА,"Україна, 63801, Харківська обл., Ізюмський р-н...",49.374251,37.623014,павленко


In [83]:
set(fops_df['Прізвище'].unique()) - set(fops_test_df['Прізвище'].unique())

{'"бенько',
 '"горошко',
 '"качуровська',
 '"коровкіна',
 '"маланчик"',
 '"сем\'янчук',
 '"тімнов',
 '"шпорт',
 'венгринович',
 'г.кліщевська',
 'григоришина',
 'заремба',
 'здорова',
 'ковенькот.',
 'корнеев',
 'мегединник',
 'михалевський',
 'мій',
 'червякова'}

In [84]:
r1 = fops_df[fops_df['Прізвище'].isin(set(fops_df['Прізвище'].unique()) - set(fops_test_df['Прізвище'].unique()))].drop_duplicates('код ЄДРПОУ')
r1.to_excel('Results/ФОПи, які звітують по залишкам, але відсутні в таблиці ЦГЗ.xlsx', index=False)
r1

,код ЄДРПОУ,Заклад,Серія препарату,Міжнародна непатентована назва,Торгівельна назва,Виробник,Термін придатності,Джерело фінансування,Кількість доз,дата оновлення,Регіон,Прізвище
886,3255920623,ФОП Григоришина,1141M019A,ХІБ,ВАКЦИНА КОН`ЮГОВАНА ДЛЯ ПРОФІЛАКТИКИ ЗАХВОРЮВА...,"СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2024-05-21,Державний бюджет 2021,11,2024-02-02,Вінницька область,григоришина
909,2335110866,ФОП Червякова,UFA22005,ГепВ,"ЕУВАКС В Вакцина для профілактики гепатиту В, ...","ЕлДжі Лайф Сайенсіс Лтд,Корея",2025-03-10,Державний бюджет 2021,6,2024-02-02,Вінницька область,червякова
1277,2673513087,ФОП Г.Кліщевська,X007788,КПК,"MMRVAXPRO II 0,5ML 10X1DOSE VIAL L25","Merck & Co., Inc.",2024-05-26,Гуманітарна допомога,37,2024-02-02,Волинська область,г.кліщевська
1492,3166913315,ФОП Михалевський Б.,0401622,АДП,"АДСОРБОВАНА ВАКЦИНА ДТ / ADSORBED DT VACCINE, ...","ПТ БІО ФАРМА (Персеро), Індонезія",2025-12-21,Державний бюджет 2021,20,2024-02-02,Волинська область,михалевський
2822,2733508291,"ФОП ""Горошко А.М.""",D2720-E1,АДП-М,"ТЕТАДІФ суспензія для ін'єкцій 0,5 мл (1 доза)...",ББ-НЦІПХЛтд. Болгарія,2026-02-28,Державний бюджет 2021,46,2024-02-02,Дніпропетровська область,"""горошко"
2848,2889703358,"ФОП ""Шпорт А.Я.""",FWL21506,Пента,"Eupenta inj.DTP-Hib-HepB 0,5mL","ЕлДжі Лайф Сайенсіс Лтд,Корея",2024-12-31,Гуманітарна допомога,55,2024-02-02,Дніпропетровська область,"""шпорт"
3490,2562701239,"ФОП ""Маланчик""",2249001B,ІПВ,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V.,2025-12-08,Гуманітарна допомога,13,2024-01-26,Донецька область,"""маланчик"""
4590,43968084,"ФОП ""Бенько Г.С.""",68D23011A,ОПВ,"Bivalent Poliomyelitis Vaccine Tipe 1&3, Live ...","Bivalent Poliomyelitis Vaccine Tipe 1&3, Live ...",2024-03-26,Гуманітарна допомога,18,2024-02-01,Івано-Франківська область,"""бенько"
4604,3334619475,ФОП Венгринович В.В.,D2708,АДП-М,"ТЕТАДІФ суспензія для ін'єкцій 0,5 мл (1 доза)...","ТЕТАДІФ суспензія для ін'єкцій 0,5 мл (1 доза)...",2025-11-30,Державний бюджет 2021,20,2024-02-01,Івано-Франківська область,венгринович
4606,,ФОП Венгринович В.В.,68D23011A,ОПВ,"Bivalent Poliomyelitis Vaccine Tipe 1&3, Live ...","Bivalent Poliomyelitis Vaccine Tipe 1&3, Live ...",2024-03-26,Гуманітарна допомога,10,2024-02-01,Івано-Франківська область,венгринович


In [85]:
set(fops_test_df['Прізвище'].unique()) - set(fops_df['Прізвище'].unique())

{'авгайтіс',
 'азарова',
 'акрітова',
 'андропова',
 'андрух',
 'андрієв',
 'апеннікова',
 'артеменко',
 'артихович',
 'арутюнян',
 'арушанян',
 'атаманюк',
 'ахмад',
 'бабаєва',
 'бабець',
 'бабінський',
 'бакликова',
 'балан',
 'балла',
 'бальжик',
 'банас',
 'баранєй',
 'барнаткевич-приходько',
 'басистюк',
 'безгін',
 'белінська',
 'бенько',
 'березнюк',
 'блищак',
 'бобир',
 'боднар',
 'бойко',
 'бондаренко',
 'бордовська',
 'бордєєва',
 'боровська',
 'брайцара',
 'бузовська',
 'буташ',
 'біла',
 'білий',
 'білоус',
 'валенко',
 'василіогло',
 'васюк',
 'ващенко',
 'ващеня',
 'венгрович',
 'вишневський',
 'возів',
 'войтович',
 'войтюк',
 'волошин',
 'волошина',
 'воробець',
 'ворона',
 'воєводіна',
 'вітюк',
 'гавриленко',
 'галайко',
 'гальчинський',
 'гамкрелідзе',
 'ганзій',
 'гардубей',
 'гаркавенко',
 'гаркавець',
 'гарькава',
 'гауліка',
 'георгієв',
 'герасій',
 'герц',
 'гетьман',
 'гладка',
 'гладуш',
 'глебчук',
 'глушко',
 'головко',
 'гончаров',
 'гончарова',
 'горець

In [86]:
r2 = fops_test_df[fops_test_df['Прізвище'].isin(set(fops_test_df['Прізвище'].unique()) - set(fops_df['Прізвище'].unique()))]
r2.to_excel('Results/ФОПи, присутні в таблиці ЦГЗ, але не звітують по залишкам.xlsx', index=False)
r2

,legal_entity_id,legal_entity_name,legal_entity_edrpou,care_type,property_type,legal_entity_email,legal_entity_website,legal_entity_phone,legal_entity_owner_name,registration_area,registration_settlement,registration_address,lat,lng,Прізвище
6,eb1b47d6-d116-446f-bfd4-d000884574ca,ЯКИМЧУК НАТАЛІЯ ВАЛЕРІЇВНА,ФОП,Первинна,ФОП,n.yakimchuk@homedoctor.in.ua,homedoctor.in.ua,+380637000640,Якимчук Наталія Валеріївна,ОДЕСЬКА,ОДЕСА,"Україна, 65000, Одеська обл., місто Одеса, ВУЛ...",46.578154,30.799197,якимчук
9,760c6b14-64ea-4886-92d6-7468860e6a63,ДОЛІШНА ОКСАНА ІВАНІВНА,ФОП,Первинна,ФОП,doleshniy.andriy@gmail.com,NaN,+380995113771,Долішна Оксана Іванівна,ІВАНО-ФРАНКІВСЬКА,ІВАНО-ФРАНКІВСЬК(З),"Україна, 76018, Івано-Франківська обл., місто ...",48.919247,24.722991,долішна
12,701bc242-68f9-4773-97e2-da36e2561c31,АРУТЮНЯН АНУШ ГНЕЛІВНА,ФОП,Первинна,ФОП,anush@homedoctor.in.ua,homedoctor.in.ua,+380734444777,Арутюнян Ануш Гнелівна,ОДЕСЬКА,ОДЕСА,"Україна, 65000, Одеська обл., місто Одеса, ВУЛ...",46.489375,30.727988,арутюнян
18,66a40717-83ec-4312-ac1e-cac5b4fd0786,ПОДПЕРИГОРА ІННА ВАСИЛІВНА,ФОП,Первинна,ФОП,i.podperygora@homedoctor.in.ua,https://homedoctor.in.ua,+380963955669,ПОДПЕРИГОРА ІННА ВАСИЛІВНА,ОДЕСЬКА,ОДЕСА,"Україна, 65020, Одеська обл., місто Одеса, вул...",46.476420,30.740399,подперигора
19,47933c79-60dc-4a69-99f6-b96c31bed482,ВИШНЕВСЬКИЙ ОЛЕКСАНДР ОЛЕКСАНДРОВИЧ,ФОП,Первинна,ФОП,nashlekarz@gmail.com,NaN,+380987798456,Вишневський Олександр Олександрович,ЖИТОМИРСЬКА,ОЛЕВСЬК,"Україна, 11001, Житомирська обл., Коростенськи...",51.215174,27.641198,вишневський
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4030,723a4b71-fe91-4461-a745-64b62951b5ce,УЛІТІНА ОЛЕНА ІВАНІВНА,ФОП,Первинна,ФОП,112ulit@gmail.com,NaN,+380669763020,Улітіна Олена Іванівна,ХЕРСОНСЬКА,НОВА КАХОВКА(З),"Україна, 74900, Херсонська обл., місто Нова Ка...",46.796690,33.470869,улітіна
4042,5258a1ba-01ed-4354-afd4-449ca3666b28,СЕМКО МАР’ЯНА БОГДАНІВНА,ФОП,Первинна,ФОП,semkomariana@gmail.com,NaN,+380978747112,Семко Мар'яна Богданівна,ІВАНО-ФРАНКІВСЬКА,ПЕРЕГІНСЬКЕ,"Україна, 77662, Івано-Франківська обл., Рожнят...",48.813750,24.193254,семко
4066,614da465-ef15-4101-960c-36cde04269d3,Осипенко Катерина Василівна,ФОП,Первинна,ФОП,katyaosypenko23@gmail.com,NaN,"+380669993632, +380669993632",Осипенко Катерина Василівна,МИКОЛАЇВСЬКА,МИКОЛАЇВ,"Україна, 54038, Миколаївська обл., місто Микол...",NaN,NaN,осипенко
4079,3d12fe6f-0715-4a9c-9e33-9ab1b72b040f,Кравець Анатолій Вікторович,ФОП,Первинна,ФОП,a.kravets73@gmail.com,NaN,+380633003410,Кравець Анатолій Вікторович,М.КИЇВ,КИЇВ,"Україна, 02140, місто Київ, вул.Гмирі Бориса, ...",NaN,NaN,кравець
